## Running prompts on OpenAI

In [5]:
%pip install markdown
%pip install annoy
%pip install openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from openai import OpenAI # OpenAI API
import json
import requests # to download some resources
import os # file operations
import numpy as np # linear algebra
import pandas as pd # data processing
from markdown import markdown # to render markdown
from IPython.display import Markdown
import annoy # Approximate Nearest Neighbors Oh Yeah for fast searching
import pickle
from annoy import AnnoyIndex

In [2]:
# read in our api key
with open('../api-keys/our_api_key.txt', 'r') as file:
    api_key = file.read().replace('\n', '')

# read in finns api key (the one we'll use for testing)
with open('../api-keys/finns_api_key.txt', 'r') as file:
    finns_api_key = file.read().replace('\n', '')

In [3]:
# Setting the OpenAI
client = OpenAI(api_key=finns_api_key) # using finns for testing

#### Sample of countries

In [4]:
# Sample of countries
sample_countries = ['Finland','Netherlands','Denmark','Norway', 'Chile', 'Bangladesh']

# Folder with raw PDFs
folder_mds = "../data/3-naps-md"

# Importing MDs files
for file in os.listdir(folder_mds):
    for country in sample_countries:
        if country in file:  
            with open(os.path.join(folder_mds, file), "r", encoding="utf-8") as md_file:
                content = md_file.read()
            globals()[country.lower()] = content # Saving the MD file in lowercase


### 1. Using the WHOLE document

In [5]:
country_test = chile
country_name = "chile"

In [6]:
system_prompt = """
You are an assistant that extracts and summarizes national action plans for antimicrobial resistance (NAPs). Your task is to answer the following questions concisely while relying strictly on the document content. 

For each question, provide:
1. A 'yes' or 'no' answer.
2. The specific text from the document that supports the answer.

Here are the questions:

1. What is the period considered for the NAP? (Provide the exact years or date range)
2. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Human Health"? (yes/no + supporting text)
3. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Animal Health"? (yes/no + supporting text)
4. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Environment"? (yes/no + supporting text)
5. Does the NAP include any strategy/policy/goal/action related to the priority sector of "Agriculture/Food Security"? (yes/no + supporting text)
6. Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools? (yes/no + supporting text)
7. Does the NAP include any specific, time-bound targets to track progress? (yes/no + supporting text)
8. Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR? (yes/no + supporting text)
9. Does the NAP establish a multisectoral committee or task force for AMR coordination? (yes/no + supporting text)
10. Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists? (yes/no + supporting text)
11. Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement? (yes/no + supporting text)
12. Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE? (yes/no + supporting text)
13. Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions? (yes/no + supporting text)
14. Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)? (yes/no + supporting text)
15. Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control? (yes/no + supporting text)
16. Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance? (yes/no + supporting text)
17. Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics? (yes/no + supporting text)
18. Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use? (yes/no + supporting text)
19. Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws? (yes/no + supporting text)
20. Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines? (yes/no + supporting text)

Format the response as a JSON object:
{
  "period": "XXXX-YYYY",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The NAP mentions specific actions in the human health sector, including..."
    },
    ...
  ]
}
"""

In [7]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    temperature=0,  # Ensures deterministic output
    top_p=0.5,  # Controls diversity
    response_format={"type": "json_object"},
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": country_test}
    ]
)

# Print the structured JSON response
print(response.choices[0].message.content)

{
  "period": "2021-2025",
  "questions": [
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?",
      "answer": "yes",
      "supporting_text": "The NAP includes strategies to prevent and control infections associated with health care and to strengthen the national surveillance system for antimicrobial resistance."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?",
      "answer": "yes",
      "supporting_text": "The NAP addresses the prevention and control of infections in animal production and includes actions to regulate the use of antimicrobials in animals."
    },
    {
      "question": "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?",
      "answer": "yes",
      "supporting_text": "The NAP emphasizes the importance of environmental factors in the emergence of an

In [8]:
import json

# Extract response content
response_json = json.loads(response.choices[0].message.content)

# Save response as JSON
output_path = f"../data/4-summary-responses-json/summary_response_{country_name}.json"
with open(output_path, "w") as f:
    json.dump(response_json, f, indent=4)

print(f"Saved response to {output_path}")

Saved response to ../data/4-summary-responses-json/summary_response_chile.json


---

### 2. Calculating performance

#### Performance: Bangladesh

In [71]:
country_name = "bangladesh"

In [72]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)
model_answers_df['question'] = model_answers_df['question'].astype(str)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

In [73]:
merged_df

,country,question,answer_gs,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,answer_llm
0,bangladesh,period_start,2017,NaN,NaN,2021
1,bangladesh,period_end,2022,NaN,NaN,2025
2,bangladesh,1,yes,NaN,NaN,yes
3,bangladesh,2,yes,NaN,NaN,yes
4,bangladesh,3,yes,NaN,NaN,yes
5,bangladesh,4,yes,NaN,NaN,yes
6,bangladesh,5,no,NaN,NaN,yes
7,bangladesh,6,no,NaN,NaN,yes
8,bangladesh,7,no,NaN,NaN,no
9,bangladesh,8,yes,NaN,NaN,yes


In [74]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

Accuracy (Yes/No Questions): 0.74
Precision (Yes/No Questions): 0.72
Recall (Yes/No Questions): 1.00
F1 Score (Yes/No Questions): 0.84
Period Extraction Accuracy: 0.00
Questions where the model failed: ['period_start' 'period_end' '5' '6' '13' '15' '19']


/var/folders/mw/rc0rz1pd0l750kf2mdl_jhbh0000gn/T/ipykernel_12737/875495442.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']


#### Performance: Netherlands

In [63]:
country_name = "netherlands"

In [64]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)
model_answers_df['question'] = model_answers_df['question'].astype(str)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

In [67]:
merged_df

,country,question,answer_gs,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,answer_llm,gs_binary,llm_binary
0,netherlands,period_start,2015,NaN,NaN,2021,NaN,NaN
1,netherlands,period_end,2019,NaN,NaN,2025,NaN,NaN
2,netherlands,1,yes,NaN,Actors agree to the following goals: The numbe...,yes,1.0,1.0
3,netherlands,2,yes,NaN,The reduction of antibiotic use has flattened....,yes,1.0,1.0
4,netherlands,3,yes,NaN,"Despite these uncertainties, the research by Z...",yes,1.0,1.0
5,netherlands,4,no,NaN,NaN,yes,0.0,1.0
6,netherlands,5,yes,NaN,We will regularly inform Parliament of the pro...,yes,1.0,1.0
7,netherlands,6,yes,NaN,The number of avoidable healthcare-associated ...,yes,1.0,1.0
8,netherlands,7,no,NaN,NaN,no,0.0,0.0
9,netherlands,8,no,NaN,NaN,yes,0.0,1.0


In [66]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

Accuracy (Yes/No Questions): 0.63
Precision (Yes/No Questions): 0.61
Recall (Yes/No Questions): 1.00
F1 Score (Yes/No Questions): 0.76
Period Extraction Accuracy: 0.00
Questions where the model failed: ['period_start' 'period_end' '4' '8' '12' '13' '15' '17' '18']


/var/folders/mw/rc0rz1pd0l750kf2mdl_jhbh0000gn/T/ipykernel_12737/875495442.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']


#### Performance: Chile

In [35]:
country_name = "chile"

In [ ]:
# Import results
with open(f"../data/4-summary-responses-json/summary_response_{country_name}.json", "r") as f:
    model_answers = json.load(f)

# Load the annotations
annotations_df = pd.read_excel('../data/5-annotations/annotations.xlsx')
annotations_df['question'] = annotations_df['question'].astype(str)
annotations_df['answer'] = annotations_df['answer'].astype(str)

# Filter for the current country
annotations_df = annotations_df[annotations_df['country'] == country_name]

# Extract period and questions from the model response
model_answers = [{"country": country_name, "question": "period_start", "answer": response_json["period"].split("-")[0]}]
model_answers.append({"country": country_name, "question": "period_end", "answer": response_json["period"].split("-")[1]})

for q in response_json["questions"]:
    model_answers.append({
        "country": country_name,
        "question": q["question"],
        "answer": q["answer"]
    })

# Convert to DataFrame
model_answers_df = pd.DataFrame(model_answers)

question_mapping = {
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Human Health'?": "1",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Animal Health'?": "2",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Environment'?": "3",
    "Does the NAP include any strategy/policy/goal/action related to the priority sector of 'Agriculture/Food Security'?": "4",
    "Does the NAP include any mechanisms for progress reporting to track how its objectives are being met, such as an annual or semi-annual progress report, a dashboard displaying the status, or similar documents/tools?": "5",
    "Does the NAP include any specific, time-bound targets to track progress?": "6",
    "Does the NAP specify any budget allocation assigned for the strategies/policies/goals/actions directed at tackling AMR?": "7",
    "Does the NAP establish a multisectoral committee or task force for AMR coordination?": "8",
    "Does the NAP include any strategy/policy/goal/action related to training and educational initiatives to increase awareness about AMR – in school curricula or professional training programs for doctors and pharmacists?": "9",
    "Does the NAP include any strategy/policy/goal/action related to public awareness campaigns in combating AMR, such as media involvement and community engagement?": "10",
    "Does the NAP include any strategy/policy/goal/action related to international collaboration and support for AMR initiatives, such as partnerships with organizations like WHO, FAO, and OIE?": "11",
    "Does the NAP include any strategy/policy/goal/action related to data collection and analysis for AMR surveillance, such as developing surveillance systems, data collection tools, setting up laboratories for AMR monitoring, or other similar actions?": "12",
    "Does the NAP include any strategy/policy/goal/action related to developing the capacity to detect and report newly emerged resistance that may constitute a public health emergency of international concern, as required by the International Health Regulations (2005)?": "13",
    "Does the NAP include any strategy/policy/goal/action related to specific prevention strategies such as sanitation, hygiene measures, and infection control?": "14",
    "Does the NAP include any strategy/policy/goal/action related to implementing the standards outlined in the OIE Terrestrial and Aquatic Animal Health Codes or the FAO/WHO Codex Alimentarius Code of Practice to Minimize and Contain Antimicrobial Resistance?": "15",
    "Does the NAP include any strategy/policy/goal/action related to regulating the distribution, prescription, and dispensation of antibiotics, such as developing or maintaining essential medicine lists for antibiotics?": "16",
    "Does the NAP include any strategy/policy/goal/action related to strengthening the legislative and regulatory framework for AMR, including laws and regulations for antimicrobial use?": "17",
    "Does the NAP include any strategy/policy/goal/action related to requiring the rational use of antibiotics through regulations/laws?": "18",
    "Does the NAP include any strategy/policy/goal/action related to participating in international collaborative research to support the development of new medicines, diagnostic tools, and vaccines?": "19"
}

model_answers_df["question"] = model_answers_df["question"].replace(question_mapping)

# Merge based on country and question
merged_df = pd.merge(annotations_df, model_answers_df, on=["country", "question"], suffixes=('_gs', '_llm'))

In [43]:
merged_df

,country,question,answer_gs,comment_input_for_protocol,supporting_text_when_true_answer_is_yes_but_prediction_is_no,answer_llm
0,chile,period_start,2021,NaN,NaN,2021
1,chile,period_end,2025,NaN,NaN,2025
2,chile,1,yes,NaN,NaN,yes
3,chile,2,yes,NaN,NaN,yes
4,chile,3,yes,NaN,NaN,yes
5,chile,4,yes,NaN,NaN,yes
6,chile,5,yes,"The LLM prediction is correct, but I think the...",NaN,yes
7,chile,6,no,NaN,NaN,yes
8,chile,7,no,NaN,NaN,no
9,chile,8,yes,NaN,NaN,yes


In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Convert categorical yes/no answers to binary (1 for yes, 0 for no)
merged_df['gs_binary'] = merged_df['answer_gs'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))
merged_df['llm_binary'] = merged_df['answer_llm'].apply(lambda x: 1 if x == 'yes' else (0 if x == 'no' else None))

# Drop rows with NaN values in either gs_binary or llm_binary for accurate calculations
yesno_df = merged_df[~merged_df['question'].isin(["period_start", "period_end"])]
yesno_df = yesno_df.dropna(subset=['gs_binary', 'llm_binary'])

# Performance for yes/no questions
accuracy = accuracy_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
precision = precision_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
recall = recall_score(yesno_df['gs_binary'], yesno_df['llm_binary'])
f1 = f1_score(yesno_df['gs_binary'], yesno_df['llm_binary'])

# Performance for period extraction (period_start, period_end)
period_matches = merged_df[merged_df['question'].isin(["period_start", "period_end"])]

# For period extraction, we don't need binary conversion, so we directly compare the answers
period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']

# Calculate period extraction accuracy as the percentage of exact matches
period_accuracy = period_matches['period_match'].mean()  # Exact match percentage

# Questions where the model failed
questions_failed = merged_df[merged_df['gs_binary'] != merged_df['llm_binary']]['question'].unique()

# Print results
print(f"Accuracy (Yes/No Questions): {accuracy:.2f}")
print(f"Precision (Yes/No Questions): {precision:.2f}")
print(f"Recall (Yes/No Questions): {recall:.2f}")
print(f"F1 Score (Yes/No Questions): {f1:.2f}")
print(f"Period Extraction Accuracy: {period_accuracy:.2f}")
print(f"Questions where the model failed: {questions_failed}")

Accuracy (Yes/No Questions): 0.79
Precision (Yes/No Questions): 0.78
Recall (Yes/No Questions): 1.00
F1 Score (Yes/No Questions): 0.88
Period Extraction Accuracy: 1.00
Questions where the model failed: ['period_start' 'period_end' '6' '11' '15' '19']


/var/folders/mw/rc0rz1pd0l750kf2mdl_jhbh0000gn/T/ipykernel_12737/875495442.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  period_matches['period_match'] = period_matches['answer_gs'] == period_matches['answer_llm']


In [24]:
# see how many tokens that last query used
response.usage

CompletionUsage(completion_tokens=1437, prompt_tokens=23295, total_tokens=24732, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))